In [1]:
#====================== Import de librerias =====================#

from pathlib import Path
import json
import gzip
from urllib.request import urlopen
import datetime
import plotly.express as px
import plotly.graph_objects as go
import wget
import logging
from tqdm import tqdm
import random
import time


import torch
import pandas as pd
import numpy as np
import csv
import os
import scipy.sparse as sp
from typing import Tuple, Dict, Any, List
from tqdm import tqdm, trange
from IPython import embed
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
import sys 

sampling_method = os.listdir()[3].split(".")[-2][3:].split("_")[-1]
old_path = os.getcwd()
os.chdir("..")
execution_path = os.getcwd()

c:\Users\brend\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
%load_ext tensorboard

logs_base_dir = "runs_"+sampling_method
os.environ["run_tensorboard"] = logs_base_dir

os.makedirs(f'{execution_path}/{"4_Modelling"}/{logs_base_dir}', exist_ok=True)
tb_fm = SummaryWriter(log_dir=f'{execution_path}/{"4_Modelling"}/{logs_base_dir}/{logs_base_dir}_FM/')
tb_rnd = SummaryWriter(log_dir=f'{execution_path}/{"4_Modelling"}/{logs_base_dir}/{logs_base_dir}_RANDOM/')

def save_data_configuration(text):
    save_data_dir = "data_config_" + sampling_method +".txt"
    path = f'{execution_path}/{"4_Modelling"}/{save_data_dir}'
    with open(path, "a") as data_file:
        data_file.write(text+"\n")

    return text

In [4]:
# Let's define some hyper-parameters
hparams = {
    'batch_size':64,
    'num_epochs':12,
    'hidden_size': 32,
    'learning_rate':1e-4,
}

# we select to work on GPU if it is available in the machine, otherwise
# will run on CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [5]:
#============ Definicion de valores de configuracion ============#

min_reviews, min_usuarios = [6,6]
col_names = {"col_id_reviewer": "reviewerID",
             "col_id_product": "asin",
             "col_unix_time": "unixReviewTime",
             "col_rating": "overall",
             "col_timestamp": "timestamp",
             "col_year": "year"}

csv_filename = execution_path/Path("3_DataPreparation/interactions_minR{}_minU{}.csv".format(min_reviews,min_usuarios))

In [6]:
df = pd.read_csv(csv_filename)
df.head()

,asin,reviewerID,overall,unixReviewTime,timestamp,year
0,0,9132,5.0,1477785600,2016-10-30 02:00:00,1970
1,0,10612,5.0,1467244800,2016-06-30 02:00:00,1970
2,0,257,1.0,1454716800,2016-02-06 01:00:00,1970
3,0,4425,5.0,1434844800,2015-06-21 02:00:00,1970
4,0,2523,4.0,1420329600,2015-01-04 01:00:00,1970


In [7]:
save_data_configuration(str(df.nunique()))
df.nunique()

asin               6178
reviewerID        14138
overall               5
unixReviewTime     3622
timestamp          3622
year                  1
dtype: int64

# Splitting dataset (TLOO strategy)

In [8]:
def split_train_test(data: np.ndarray,
                     n_users: int) -> Tuple[np.ndarray, np.ndarray]:
    # Split and remove timestamp
    train_x, test_x = [], []
    for u in trange(n_users, desc='spliting train/test and removing timestamp...'):
        user_data = data[data[:, 0] == u]
        sorted_data = user_data[user_data[:, -1].argsort()]
        if len(sorted_data) == 1:
            train_x.append(sorted_data[0][:-1])
        else:
            train_x.append(sorted_data[:-1][:, :-1])
            test_x.append(sorted_data[-1][:-1])
    return np.vstack(train_x), np.stack(test_x)

In [9]:
data = df[[*col_names.values()][:3]].astype('int32').to_numpy()
data

array([[      9132,          0, 1477785600],
       [     10612,          0, 1467244800],
       [       257,          0, 1454716800],
       ...,
       [      9051,       6177, 1530144000],
       [      3412,       6177, 1527465600],
       [      9805,       6177, 1527206400]])

In [10]:
add_dims=0
for i in range(data.shape[1] - 1):  # do not affect to timestamp
    # MAKE IT START BY 0
    data[:, i] -= np.min(data[:, i])
    # RE-INDEX
    data[:, i] += add_dims
    add_dims = np.max(data[:, i]) + 1
dims = np.max(data, axis=0) + 1
print("Dim of users: {}\nDim of items: {}\nDims of unixtime: {}".format(dims[0], dims[1], dims[2]))
data

Dim of users: 14138
Dim of items: 20316
Dims of unixtime: 1538006401


array([[      9132,      14138, 1477785600],
       [     10612,      14138, 1467244800],
       [       257,      14138, 1454716800],
       ...,
       [      9051,      20315, 1530144000],
       [      3412,      20315, 1527465600],
       [      9805,      20315, 1527206400]])

In [11]:
train_x, test_x = split_train_test(data, dims[0])
train_x

spliting train/test and removing timestamp...: 100%|██████████| 14138/14138 [00:04<00:00, 3003.51it/s]


array([[    0, 19248],
       [    0, 19249],
       [    0, 14823],
       ...,
       [14137, 14159],
       [14137, 18245],
       [14137, 18904]])

# Negative sampling

In [12]:
train_x = train_x[:, :2]
dims = dims[:2]
print("New dims:",dims)
print("New train_x:\n",train_x)

New dims: [14138 20316]
New train_x:
 [[    0 19248]
 [    0 19249]
 [    0 14823]
 ...
 [14137 14159]
 [14137 18245]
 [14137 18904]]


In [13]:
def build_adj_mx(n_feat:int, data:np.ndarray) -> sp.dok_matrix :
    train_mat = sp.dok_matrix((n_feat, n_feat), dtype=np.float32)
    for x in tqdm(data, desc=f"BUILDING ADJACENCY MATRIX..."):
        train_mat[x[0], x[1]] = 1.0
        train_mat[x[1], x[0]] = 1.0
        # IDEA: We treat features that are not user or item differently because we do not consider
        #  interactions between contexts
        if data.shape[1] > 2:
            for idx in range(len(x[2:])):
                train_mat[x[0], x[2 + idx]] = 1.0
                train_mat[x[1], x[2 + idx]] = 1.0
                train_mat[x[2 + idx], x[0]] = 1.0
                train_mat[x[2 + idx], x[1]] = 1.0
    return train_mat

In [14]:
def ng_sample(data: np.ndarray, dims: list, num_ng:int=4) -> Tuple[np.ndarray, sp.dok_matrix]:
    rating_mat = build_adj_mx(dims[-1], data)
    interactions = []
    min_item, max_item = dims[0], dims[1]
    for num, x in tqdm(enumerate(data), desc='perform negative sampling...'):
        interactions.append(np.append(x, 1))
        for t in range(num_ng):
            j = np.random.randint(min_item, max_item) #if not pop else random.sample(items_to_sample, 1)[0]
            # IDEA: Loop to exclude true interactions (set to 1 in adj_train) user - item
            while (x[0], j) in rating_mat or j == int(x[1]):
                j = np.random.randint(min_item, max_item) #if not pop else random.sample(items_to_sample, 1)[0]
            interactions.append(np.concatenate([[x[0], j], x[2:], [0]]))
    return np.vstack(interactions), rating_mat

In [15]:
train_x, rating_mat = ng_sample(train_x, dims)
print("Dimensions matrix:\n",dims)
print("\nRating matrix:")
rating_mat

BUILDING ADJACENCY MATRIX...: 100%|██████████| 123226/123226 [00:02<00:00, 41095.29it/s]
perform negative sampling...: 123226it [00:05, 20778.86it/s]


Dimensions matrix:
 [14138 20316]

Rating matrix:


<20316x20316 sparse matrix of type '<class 'numpy.float32'>'
	with 246452 stored elements in Dictionary Of Keys format>

In [16]:
dims[-1]-dims[0]

6178

In [17]:
# Exercise 2

## Evaristo
#### number of ones
print(np.count_nonzero(rating_mat.toarray())/(dims[-1]*dims[-1]))
### number of zeros
print(1 - np.count_nonzero(rating_mat.toarray())/(dims[-1]*dims[-1]))

# ## Brenda
# #### Who sparse is the matrix??
# print(1 - rating_mat.shape[0] / rating_mat.count_nonzero())

0.000597112191656141
0.9994028878083439


In [18]:
train_x[:10]

array([[    0, 19248,     1],
       [    0, 16502,     0],
       [    0, 18683,     0],
       [    0, 15512,     0],
       [    0, 15043,     0],
       [    0, 19249,     1],
       [    0, 17742,     0],
       [    0, 15265,     0],
       [    0, 18764,     0],
       [    0, 15591,     0]])

# Creating dataset class

In [19]:
class PointData(Dataset):
    def __init__(self,
                 data: np.ndarray,
                 dims: list) -> None:
        """
        Dataset formatter adapted point-wise algorithms
        Parameters
        """
        super(PointData, self).__init__()
        self.interactions = data
        self.dims = dims

    def __len__(self) -> int:
        return len(self.interactions)
        
    def __getitem__(self, 
                    index: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Return the pairs user-item and the target.
        """
        return self.interactions[index][:-1], self.interactions[index][-1]

train_dataset = PointData(train_x, dims)

In [20]:
train_dataset[0]

(array([    0, 19248]), 1)

# Preparing the test set for inference

In [21]:
test_x

array([[    0, 17249],
       [    1, 18015],
       [    2, 14196],
       ...,
       [14135, 19938],
       [14136, 20214],
       [14137, 15542]])

In [22]:
import math
print(rating_mat.shape)
bits = math.ceil(math.log(rating_mat.shape[0],2))
print("rating_mat contains log2(rating_mat.shape[0]) = {} bits".format(bits))

(20316, 20316)
rating_mat contains log2(rating_mat.shape[0]) = 15 bits


In [23]:
def create_test_no_interactions_stratified_items(train_x: np.ndarray, test_x: np.ndarray, dims, num_samples):
    save_data_configuration("\n"+"#"*4+"  zero_positions: Stratified Sampling By Items  "+"#"*4)

    seed = 2   # get same results
    random.seed(a = seed)

    items_test = np.unique(test_x[:, 1])
    total_users = range(0,dims[0])
    zero_positions = np.zeros((num_samples * len(items_test), 2)).astype(int)
    start_index = 0

    for item in tqdm(items_test):
        strata_users_train = np.unique(train_x[train_x[:,1] == item][:,0])
        strata_users_test = np.unique(test_x[test_x[:,1] == item][:,0])
        random_selection_users = random.choices(list(set(total_users) - set(strata_users_train) - set(strata_users_test)), k=num_samples) # random.sample

        zero_positions[start_index:start_index + num_samples, 0] = random_selection_users
        zero_positions[start_index:start_index + num_samples, 1] = item
        start_index += num_samples

    indexes = np.argsort(zero_positions[:,0])
    # [00:14<00:00, 325.18it/s]
    zero_positions[:,0] = zero_positions[indexes,0]
    zero_positions[:,1] = zero_positions[indexes,1]
    return zero_positions
    # [00:14<00:00, 310.87it/s]
    # return np.vstack([zero_positions[indexes,0],zero_positions[indexes,1]]).T

In [24]:
# def create_test_no_interactions_stratified_itemss(train_x: np.ndarray, test_x: np.ndarray, dims_usuarios_productos: Tuple[int, int],  num_samples: int) -> np.ndarray:
#     """
#     Esta funcion se encarga de crear de manera eficiente un dataset que contenga las interacciones usuario-producto en test que no se hayan producido.
    
#     Argumentos:
#         train_x (np.ndarray): matriz de entrenamiento con las interacciones usuario-producto previas
#         test_x (np.ndarray): matriz de prueba con las interacciones usuario-producto previas
#         dims_usuarios_productos (Tuple[int, int]): rango de productos y usuarios disponibles
    
#     Retorno:
#         np.ndarray: una matriz con todas las interacciones usuario-producto en test que no se hayan producido
#     """
#     from tqdm import tqdm
#     import random
    
#     # Identificamos los usuarios presentes en la prueba
#     usuarios_test = np.unique(test_x[:, 0])
#     # Identificamos el rango de productos disponibles
#     total_productos = range(dims_usuarios_productos[0]-1, dims_usuarios_productos[1])
    
#     # Recorremos cada usuario presente en la prueba
#     for usuario in tqdm(usuarios_test):
#         productos_train = np.unique(train_x[train_x[:, 0] == usuario][:, 1])
#         productos_a_machear = random.choices(list(set(total_productos) - set(productos_train)), k=num_samples)
#         lista_por_usuario = np.vstack([(np.ones(len(productos_a_machear))*usuario).astype(int), productos_a_machear]).T
        
#         # Si es el primer usuario, inicializamos una matriz con sus interacciones
#         if usuario == 0:
#             zero_positions = lista_por_usuario
#         # Si no es el primer usuario, concatenamos sus interacciones a la matriz existente
#         else:
#             zero_positions = np.concatenate((zero_positions, lista_por_usuario), axis=0)
            
#     return zero_positions

In [25]:
# def zero_positions_mode(mode, rating_mat, train_x, test_x, dims):
   
#     if mode == 0:
#         save_data_configuration("\n"+"#"*4+"  zero_positions: all data  "+"#"*4)
#         return np.asarray(np.where(rating_mat.A==0)).T
#     elif mode == 1:
#         zero_true_matrix = np.where(rating_mat.A==0)
#         save_data_configuration("\n"+"#"*4+"  zero_positions: all data separated by rows  "+"#"*4)
#         return np.asarray([zero_true_matrix[0],zero_true_matrix[1]]).T
#     else:
#         save_data_configuration("\n"+"#"*4+"  zero_positions: random sampling  "+"#"*4)
#         zp = create_test_no_interactions(train_x, test_x, dims,  num_samples=199)
#         return zp

In [26]:

num_samples = 199 # number of users in strata
zero_positions = create_test_no_interactions_stratified_items(train_x, test_x, dims,num_samples)
print(save_data_configuration(str(zero_positions.shape)+"\n"))
zero_positions

100%|██████████| 4654/4654 [00:13<00:00, 346.89it/s]


(926146, 2)



array([[    0, 19191],
       [    0, 14411],
       [    0, 16438],
       ...,
       [14137, 17445],
       [14137, 19291],
       [14137, 14205]])

In [27]:
"""
Esta parte del código se usaba porque el zero positions venia de la rating matrix la cual tenia informacion no útil como (user,user) o (item,item), por eso nos quedábamos con items mayores a dims[0]
Ya que si recordamos, la matrix tenia size: (users + items)
rango items iniciales = 0 ... 6177
rango items actuales  = 14138 ... 20135
"""

# items2compute = []
# for user in trange(dims[0]):
#     aux = zero_positions[zero_positions[:, 0] == user][:, 1]
#     items2compute.append(aux[aux >= dims[0]])
# items2compute[0]


items2compute = []
for user in trange(dims[0]):
    # aux = np.array(sorted(zero_positions[start:start+num_samples,1]))
    # [00:36<00:00, 386.85it/s]
    aux = np.array(sorted(zero_positions[zero_positions[:, 0] == user][:, 1]))
    items2compute.append(aux.copy())
items2compute[0]

100%|██████████| 14138/14138 [00:38<00:00, 369.55it/s]


array([14158, 14170, 14182, 14217, 14276, 14302, 14395, 14411, 14537,
       14675, 14755, 14805, 14856, 14887, 14989, 15140, 15173, 15251,
       15309, 15376, 15588, 15619, 15646, 15819, 15860, 15893, 15916,
       16019, 16145, 16153, 16438, 16495, 16745, 16769, 16838, 16925,
       16939, 16964, 16994, 17243, 17500, 17586, 17595, 17687, 17720,
       17760, 17819, 17924, 18034, 18300, 18369, 18498, 18539, 18553,
       18562, 18619, 18936, 18994, 18999, 19082, 19190, 19191, 19286,
       19319, 19370, 19375, 19380, 19397, 19510, 19527, 19533, 19550,
       19659, 19712, 19811, 19997, 20092, 20229, 20232, 20296])

In [28]:
def build_test_set(itemsnoninteracted:list, gt_test_interactions: np.ndarray) -> list:
    #max_users, max_items = dims # number users (943), number items (2625)
    test_set = []
    for pair, negatives in tqdm(zip(gt_test_interactions, itemsnoninteracted), desc="BUILDING TEST SET..."):
        # APPEND TEST SETS FOR SINGLE USER
        negatives = np.delete(negatives, np.where(negatives == pair[1]))
        single_user_test_set = np.vstack([pair, ] * (len(negatives)+1))
        single_user_test_set[:, 1][1:] = negatives
        test_set.append(single_user_test_set.copy())
    return test_set

print(test_x[0])
test_x = build_test_set(items2compute, test_x)
test_x[0]

[    0 17249]


BUILDING TEST SET...: 14138it [00:01, 7885.74it/s]


array([[    0, 17249],
       [    0, 14158],
       [    0, 14170],
       [    0, 14182],
       [    0, 14217],
       [    0, 14276],
       [    0, 14302],
       [    0, 14395],
       [    0, 14411],
       [    0, 14537],
       [    0, 14675],
       [    0, 14755],
       [    0, 14805],
       [    0, 14856],
       [    0, 14887],
       [    0, 14989],
       [    0, 15140],
       [    0, 15173],
       [    0, 15251],
       [    0, 15309],
       [    0, 15376],
       [    0, 15588],
       [    0, 15619],
       [    0, 15646],
       [    0, 15819],
       [    0, 15860],
       [    0, 15893],
       [    0, 15916],
       [    0, 16019],
       [    0, 16145],
       [    0, 16153],
       [    0, 16438],
       [    0, 16495],
       [    0, 16745],
       [    0, 16769],
       [    0, 16838],
       [    0, 16925],
       [    0, 16939],
       [    0, 16964],
       [    0, 16994],
       [    0, 17243],
       [    0, 17500],
       [    0, 17586],
       [   

# Building Factorization Machines model

In [29]:
class FM_operation(torch.nn.Module):

    def __init__(self, 
                 reduce_sum: bool=True) -> None:
        super().__init__()
        self.reduce_sum = reduce_sum

    def forward(self,
                x: torch.Tensor) -> float:
        """
        :param x: Float tensor of size ``(batch_size, num_fields, embed_dim)``
        """
        # square_of_sum = np.sum(x, dim=1) ** 2 # ...
        # sum_of_square = np.sum(x ** 2, dim=1) # ...
        
        square_of_sum = torch.pow(torch.sum(x, dim=1),2)
        sum_of_square = torch.sum(torch.pow(x,2), dim=1)
        ix = square_of_sum - sum_of_square
        if self.reduce_sum:
            ix = torch.sum(ix, dim=1, keepdim=True)
        return 0.5 * ix
        

In [30]:
class FactorizationMachineModel(torch.nn.Module):
    """
    A pytorch implementation of Factorization Machine.

    Reference:
        S Rendle, Factorization Machines, 2010.
    """

    def __init__(self, 
                 field_dims: list,
                 embed_dim: float) -> None:
        super().__init__()
        self.linear = torch.nn.Linear(len(field_dims), 1)
        self.embedding = torch.nn.Embedding(field_dims[-1], embed_dim)
        self.fm = FM_operation(reduce_sum=True)

        torch.nn.init.xavier_uniform_(self.embedding.weight.data)

    def forward(self, interaction_pairs: torch.Tensor) -> torch.Tensor:
        """
        :param interaction_pairs: Long tensor of size ``(batch_size, num_fields)``
        """
        out = self.linear(interaction_pairs.float()) + self.fm(self.embedding(interaction_pairs))
        return out.squeeze(1)
        
    def predict(self, 
                interactions: np.ndarray,
                device: torch.device) -> torch.Tensor:
        # return the score, inputs are numpy arrays, outputs are tensors
        test_interactions = torch.from_numpy(interactions).to(dtype=torch.long, device=device) #, dtype=torch.long)
        output_scores = self.forward(test_interactions)
        return output_scores

# Pipeline functions

## Training

In [31]:
from statistics import mean

def train_one_epoch(model: torch.nn.Module,
                    optimizer: torch.optim,
                    data_loader: torch.utils.data.DataLoader,
                    criterion: torch.nn.functional,
                    device: torch.device) -> float:
    model.train()
    total_loss = []

    for i, (interactions, targets) in enumerate(data_loader):
        interactions = interactions.to(device)
        targets = targets.to(device)

        predictions = model(interactions)
    
        loss = criterion(predictions, targets.float())
        model.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss.append(loss.item())

    return mean(total_loss)

# Define metrics

In [32]:
import math

def getHitRatio(recommend_list: list,
                gt_item: int) -> bool:
    if gt_item in recommend_list:
        return 1
    else:
        return 0

def getNDCG(recommend_list: list,
            gt_item: int) -> float:
    idx = np.where(recommend_list == gt_item)[0]
    if len(idx) > 0:
        return math.log(2)/math.log(idx+2)
    else:
        return 0

# Inference


In [33]:
def test(model: torch.nn.Module,
         test_x: np.ndarray,
         device: torch.device,
         topk: int=10) -> Tuple[float, float]:
    # Test the HR and NDCG for the model @topK
    model.eval()

    HR, NDCG = [], []
    for user_test in test_x:
        gt_item = user_test[0][1]
        predictions = model.predict(user_test, device)
        _, indices = torch.topk(predictions, topk)
        recommend_list = user_test[indices.cpu().detach().numpy()][:, 1]

        HR.append(getHitRatio(recommend_list, gt_item))
        NDCG.append(getNDCG(recommend_list, gt_item))
    return mean(HR), mean(NDCG)

# PIPELINE
## Defining the model, the loss and the optimizer



In [34]:
dims = train_dataset.dims
model = FactorizationMachineModel(dims, hparams['hidden_size']).to(device)

criterion = torch.nn.BCEWithLogitsLoss(reduction='mean')
optimizer = torch.optim.Adam(params=model.parameters(), lr=hparams['learning_rate'])

## Random evaluation

In [35]:
import random
class RandomModel(torch.nn.Module):
    def __init__(self, 
                 dims: list) -> None:
        super(RandomModel, self).__init__()
        """
        Simple random based recommender system
        """
        self.all_items = list(range(dims[0], dims[1]))

    def forward(self) -> None:
        pass

    def predict(self,
                interactions: np.ndarray,
                device=None) -> torch.Tensor:
        return torch.FloatTensor(random.sample(self.all_items, len(interactions)))

rnd_model = RandomModel(dims)

## Final pipeline

In [36]:
data_loader = DataLoader(train_dataset, batch_size=hparams['batch_size'], shuffle=True, num_workers=0)

# Start training the model

In [37]:
# DO EPOCHS NOW
topk = 10
for epoch_i in range(hparams['num_epochs']):
    #data_loader.dataset.negative_sampling()
    train_loss = train_one_epoch(model, optimizer, data_loader, criterion, device)
    hr, ndcg = test(model, test_x, device, topk=topk)
    
    print(save_data_configuration(f'MODEL: FACTORIZATION MACHINE'))
    print(save_data_configuration(f'epoch {epoch_i}:'))
    print(save_data_configuration(f'training loss = {train_loss:.4f} | Eval: HR@{topk} = {hr:.4f}, NDCG@{topk} = {ndcg:.4f} '))
    print('\n')
 
    tb_fm.add_scalar('train/loss', train_loss, epoch_i)
    tb_fm.add_scalar('eval/HR@{topk}', hr, epoch_i)
    tb_fm.add_scalar('eval/NDCG@{topk}', ndcg, epoch_i)

    hr, ndcg = test(rnd_model, test_x, device, topk=topk)

    print(save_data_configuration(f'MODEL: RANDOM'))
    print(save_data_configuration(f'epoch {epoch_i}:'))
    print(save_data_configuration(f'training loss = {train_loss:.4f} | Eval: HR@{topk} = {hr:.4f}, NDCG@{topk} = {ndcg:.4f} '))
    print('\n')
 

    tb_rnd.add_scalar('eval/HR@{topk}', hr, epoch_i)
    tb_rnd.add_scalar('eval/NDCG@{topk}', ndcg, epoch_i)

MODEL: FACTORIZATION MACHINE
epoch 0:
training loss = 1.3780 | Eval: HR@10 = 0.1326, NDCG@10 = 0.0603 


MODEL: RANDOM
epoch 0:
training loss = 1.3780 | Eval: HR@10 = 0.1520, NDCG@10 = 0.0688 


MODEL: FACTORIZATION MACHINE
epoch 1:
training loss = 0.5134 | Eval: HR@10 = 0.1490, NDCG@10 = 0.0834 


MODEL: RANDOM
epoch 1:
training loss = 0.5134 | Eval: HR@10 = 0.1540, NDCG@10 = 0.0698 


MODEL: FACTORIZATION MACHINE
epoch 2:
training loss = 0.5054 | Eval: HR@10 = 0.1791, NDCG@10 = 0.1047 


MODEL: RANDOM
epoch 2:
training loss = 0.5054 | Eval: HR@10 = 0.1534, NDCG@10 = 0.0701 


MODEL: FACTORIZATION MACHINE
epoch 3:
training loss = 0.4873 | Eval: HR@10 = 0.2679, NDCG@10 = 0.1566 


MODEL: RANDOM
epoch 3:
training loss = 0.4873 | Eval: HR@10 = 0.1526, NDCG@10 = 0.0691 


MODEL: FACTORIZATION MACHINE
epoch 4:
training loss = 0.4641 | Eval: HR@10 = 0.3178, NDCG@10 = 0.1840 


MODEL: RANDOM
epoch 4:
training loss = 0.4641 | Eval: HR@10 = 0.1535, NDCG@10 = 0.0692 


MODEL: FACTORIZATION MACH

# Visualization

In [ ]:
!kill 2820

"kill" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [41]:
# 'c:\\Users\\brend\\OneDrive\\Escritorio\\Postgrado\\RecSys_Project\\GitHub_repo\\Recommender-System-2023\\Implementation'
os.chdir(os.getcwd() / Path("4_Modelling"))


In [52]:
%tensorboard --logdir run_tensorboard

Reusing TensorBoard on port 6006 (pid 21132), started 0:00:15 ago. (Use '!kill 21132' to kill it.)